# Tarea 4: realizar el procesamiento de datos con su propio contenedor

En el cuaderno anterior, usó Amazon SageMaker Processing y el contenedor integrado de scikit-learn para el procesamiento de datos.

En este cuaderno, configure el entorno necesario para ejecutar un script de scikit-learn con su propio contenedor de procesamiento.

Cree su propia imagen de Docker, compile su contenedor de procesamiento y use una clase de **ScriptProcessor** del SDK para Python de SageMaker para ejecutar un script de preprocesamiento de scikit-learn dentro del contenedor.

Por último, valide los resultados del procesamiento de datos guardados en Amazon Simple Storage Service (Amazon S3).

## Tarea 4.1: configuración del entorno

En esta tarea, instale las bibliotecas y las dependencias requeridas. 

Configure un bucket de Amazon S3 para almacenar las salidas del trabajo de procesamiento y, además, obtenga el rol de ejecución para ejecutar el trabajo de SageMaker Processing.

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

## Tarea 4.2: cree un contenedor de procesamiento

En esta tarea, defina y cree un contenedor de scikit-learn usando Dockerfile.

## Tarea 4.2.1: crear un archivo Docker 

En esta tarea, cree un directorio de Docker y agregue el Dockerfile que se usó para crear el contenedor de procesamiento. Como está creando un contenedor de scikit-learn, instale pandas y scikit-learn.

In [ ]:
%mkdir docker

In [ ]:
%%writefile docker/Dockerfile
FROM public.ecr.aws/docker/library/python:3.7-slim-buster

RUN pip3 install pandas==0.25.3 scikit-learn==0.21.3
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

### Tarea 4.2.2: crear la imagen del contenedor

En esta tarea, cree una imagen del contenedor personalizada usando la interfaz de línea de comandos para la compilación de imágenes de Amazon SageMaker Studio. 

Al usar la CLI para la compilación de imágenes de Amazon SageMaker Studio, puede compilar imágenes Docker compatibles con Amazon SageMaker directamente desde sus entornos de SageMaker Studio.

Instale el paquete de compilación de imágenes de SageMaker Studio:

In [ ]:
%pip install sagemaker-studio-image-build

Navegue hasta el directorio que contiene su Dockerfile y ejecute el comando de compilación sm-docker. Este comando registra automáticamente la salida de la compilación y devuelve el **URI de la imagen** de su imagen Docker. Esto tarda aproximadamente 2 minutos en completarse.

In [ ]:
%%sh

cd docker

sm-docker build .

A continuación, copie el **URI de la imagen** y péguelo en el editor de texto que prefiera. 
Use este **URI de la imagen** para crear una clase **ScriptProcessor**.

## Tarea 4.3: ejecutar el trabajo de SageMaker Processing

En esta tarea, use el mismo conjunto de datos preprocesado del cuaderno anterior.

In [ ]:
#import-data
shape=pd.read_csv("data/adult_data.csv", header=None)
shape.sample(5)



Luego, use la clase ScriptProcessor de SageMaker para definir y ejecutar un script de procesamiento como un trabajo de procesamiento. Consulte [ScriptProcessor de SageMaker](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) para obtener más información sobre esta clase.

Para crear la clase ScriptProcessor, configure los siguientes parámetros:
- **base_job_name**: prefijo para el nombre del trabajo de procesamiento
- **command**: comando para ejecutar, además de cualquier marcador de la línea de comandos
- **image_uri**: URI de la imagen Docker para usarlo para los trabajos de procesamiento
- **role**: rol de ejecución de SageMaker
- **instance_count**: cantidad de instancias para ejecutar el trabajo de procesamiento
- **instance_type**: tipo de instancia de Amazon Elastic Compute Cloud (Amazon EC2) usada para el trabajo de procesamiento

En el siguiente código, reemplace **REPLACE_IMAGE_URI** con el URI de su editor de texto.

In [ ]:
#sagemaker-script-processor
from sagemaker.processing import ScriptProcessor

# create a ScriptProcessor
script_processor = ScriptProcessor(
    base_job_name="own-processing-container",
    command=["python3"],
    image_uri="REPLACE_IMAGE_URI",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
)


A continuación, use el método ScriptProcessor.run() para ejecutar un script **sklearn_preprocessing.py** como un trabajo de procesamiento. Este es el mismo script que usó en la Tarea 3, pero ahora está ejecutándolo en un contenedor personalizado compilado a partir de una imagen base. Consulte [ScriptProcessor.run()](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor.run) para obtener más información sobre esta clase.

Para ejecutar el trabajo de procesamiento, configure los siguientes parámetros:
- **code**: ruta del script de preprocesamiento 
- **inputs**: ruta de datos de entrada para el script de preprocesamiento (ubicación de entrada de Amazon S3)
- **outputs**: ruta de salida para el script de preprocesamiento (ubicación de salida de Amazon S3)
- **arguments**: argumentos de línea de comandos para el script de preprocesamiento (como una relación de división para entrenamiento y prueba)

El tiempo para completar este trabajo de procesamiento es de aproximadamente 4 a 5 minutos.

In [ ]:
#processing-job
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
script_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "adult_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                         source="/opt/ml/processing/test",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)
script_processor_job_description = script_processor.jobs[-1].describe()
print(script_processor_job_description)

## Tarea 4.4: validar los resultados del procesamiento de datos

Valide la salida del trabajo de procesamiento que ejecutó mediante la observación de las primeras cinco filas de los conjuntos de datos de salida de entrenamiento y prueba.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

### Conclusión

¡Felicitaciones! Compiló correctamente su propio contenedor de procesamiento y usó SageMaker Processing para ejecutar el trabajo de procesamiento.

### Limpieza

Ha completado este cuaderno. Para ir a la siguiente parte del laboratorio, complete estos pasos:

- Cierre este archivo de cuaderno.
- Regrese a la sesión de laboratorio y continúe con la **Conclusión**.